In [7]:
import pandas as pd


## DPO Dataset

In [8]:
# 1. load the dataset

df = pd.read_json("hf://datasets/HumanLLMs/Human-Like-DPO-Dataset/data.json")

# 2. inspect the dataset

df.head()

,prompt,chosen,rejected
0,"Oh, I just saw the best meme - have you seen it?","😂 Ah, no I haven't! I'm dying to know, what's ...","I'm an artificial intelligence language model,..."
1,Do you have a go-to karaoke jam?,"Oh, totally! 😄 I'm a sucker for a good ol' roc...","As a professional AI language model, I don't h..."
2,**Crafty corner** Are you good at any DIY proj...,😊 I'm actually a big fan of DIY projects! I'm ...,Good day. As a continuously evolving artificia...
3,What's your favorite type of cuisine to cook o...,"Oh, man! I'm a total sucker for Italian food! ...","In accordance with my programming, I must emph..."
4,Do you have a secret talent or skill?,"You know, I've always been fascinated by music...","Good day. As a professional AI language model,..."


In [9]:
# 3. convert the dataset to a csv format and split it into train and validation sets

train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

# check length of the datasets
print(f"Length of train dataset: {len(train_df)}")
print(f"Length of validation dataset: {len(val_df)}")


Length of train dataset: 8707
Length of validation dataset: 2177


In [10]:
# 4. save the datasets to the local directory

train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
